In [15]:
'''Import third-party libraries'''
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
'''Downloading training data from open datasets'''
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = False,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = False,
    transform = ToTensor()
)

In [ ]:
'''Data for training preparation'''

batch_size = 64
# Create data loaders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader =  DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"shape of X:{X.shape}")
    print(f"shape and type of y:{y.shape} {y.dtype}")
    break

In [ ]:
'''Creaing models'''
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_avaiable() else "cpu"
print(f"Using {device} device") 

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        '''Define the layers of the network'''
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        '''Specify how data will pass through the network'''
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Instantiate a model 
model = NeuralNetwork().to(device)
print(model)

AttributeError: module 'torch' has no attribute 'accelerator'

In [ ]:
'''Define loss function and optimizer'''
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

#1. model.parameters()会返回模型所有可学习参数（就是weights和bias）的一个iterator。这些参数是torch.nn.Parameter
# 的实例，是require_gradients = True，并且在训练过程中会更新的参数。
# 如果想要打印模型的参数，可以使用如下的方式：

print("="*50)
print(model.parameters())
print("="*50)

params = list(model.parameters())

# Print the number of parameter tensors
print(f"Number of parameter tensors: {len(params)}")

# Print each parameter's shape
for param in params:
    print(f"Parameter {param.name}: Shape = {param.shape}")

In [ ]:
'''Define the model training process'''
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) #dataloader.dataset will get the original dataset object, and we 
    # can directly access the dataset through “dataloader.dataset”，比如dataloader.dataset[0]访问第一条数据
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y) #计算出来的是一个 PyTorch tensor with additional attributes and functions

        #Backpropagation
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()

        # print loss
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss:{loss:>7f} [{current:>5d} / {size:>5d}]")

# Note:
# 1.enumerate(dataloader)会返回一个mini-batch（在实例化DataLoader对象的时候设置的batch_size）的index和这个mini-batch里的数据；


# 1.loss常用的和函数有：
![属性](./images/loss常用属性.jpg)<br>
![函数](./images/loss常用函数.jpg)<br>

# 2.For the optimizer:
## 2.1why using optimizer.zero_grad() to clear gradients before next iteration ? 
&nbsp;&nbsp;&nbsp;&nbsp;因为在pytorch中，默认是梯度的计算是累积的，这就意味着每次调用loss.backward()，那么计算出来的梯度就会被累加到模型的参数的.grad属性上，而非直接覆盖原有.grad的值；因此，如果不在下一次迭代前清除梯度的话，梯度就会持续累积，导致模型的梯度更新出错；<br>
&nbsp;&nbsp;&nbsp;&nbsp;而在pytorch中，为什么又要默认支持梯度是累积的呢？<br>
&nbsp;&nbsp;&nbsp;&nbsp;一方面是如果需要在多个batches之后才更新梯度的情况下，支持梯度累加之后更新，比如需要训练很大的模型，这个模型无法放到内存中？<br>
&nbsp;&nbsp;&nbsp;&nbsp;另一方面是为了支持有多个loss的情况下，多次调用了loss.backward()函数计算梯度，允许最后累加起来更新参数。<br>

## 2.2 针对训练大模型，需要使用累加梯度的情况的进一步说明
（1）为什么训练大模型会需要使用累加梯度？
&nbsp;&nbsp;&nbsp;&nbsp;因为训练大模型的时候，通常伴随着很大的batch_size， 并且model_size会很大，就会很容易造成显存OOM的问题；<br>
&nbsp;&nbsp;&nbsp;&nbsp;但是模型大小不能随意变小，而骤降batch_size会导致训练不稳定，收敛速度缓慢的问题，这个时候就可以考虑使用累加梯度，具体的使用方式的一个例子如下所示：<br>
```python
accumulation_steps = 4  # Accumulate gradients over 4 mini-batches

for i, (x, y) in enumerate(dataloader):
    optimizer.zero_grad() if i % accumulation_steps == 0 else None  # Clear grads only at start

    pred = model(x)  # Forward pass
    loss = loss_fn(pred, y)  # Compute loss
    loss.backward()  # Compute gradients (accumulate)

    if (i + 1) % accumulation_steps == 0:  
        optimizer.step()  # Update weights only after 4 mini-batches
        optimizer.zero_grad()  # Clear gradients for next accumulation cycle
```
&nbsp;&nbsp;&nbsp;&nbsp;假设原始不使用累积梯度的时候，batch_size = 64，每个batch都计算并且更新参数。保持模型大小不变，这个时候会爆显存，是因为pred = model(x)，模型在前向传播得到计算图的过程中会OOM。这个时候我们就可以把batch_size = 16，缩小为原来的四分之一，保证在一次前向传播的时候不会OOM，然后设置accumulation_steps = 4, 每个batch都计算梯度，但是每4个batch才一起更新参数，这样两者乘起来就是64，效果和原来设置batch_size = 64是一样的效果。


## 2.3 optimier常用的属性和函数：
![属性](./images/optimizer常用属性.jpg)<br>
![函数](./images/optimizer常用函数.jpg)<br>


# 3.关于model.train()和model.eval()的区别：
## 3.1 两者的区别：
![区别](./images/训练和验证模式的主要区别.jpg)
## 3.2 这里BatchNorm使用的batch/learned statistics又是什么呢？<br>
&nbsp;&nbsp;&nbsp;&nbsp;(1)batch statistics指的是在训练过程中，每个batch中计算的均值和方差，在不同的batch中，计算的数值是不一样的，因为每个batch的数据也是不一样的；<br>
&nbsp;&nbsp;&nbsp;&nbsp;(2)learned statistics指的是使用在训练过程中，经过使用batch中计算的数据，不断更新得到的全局Running Mean & Variance，训练结束之后，就得得到了learned statistics.<br>
&nbsp;&nbsp;&nbsp;&nbsp;(3)对于两者具体的计算公式以及区别如下：<br>
![batch_statistics计算公式](./images/batch_statistics计算公式.jpg)<br>
![learned_statistics计算公式](./images/learned_statistics计算公式.jpg)<br>
![batch_learned_statistics区别](./images/batch_learned_statistics区别.jpg)<br>

# 4.model常用的函数和属性有哪些？
![model常用的属性](./images/model常用的属性.jpg)<br>
![model常用的函数](./images/model常用的函数.jpg)<br>
注意：model.save_state_dict()函数是不存在的，如果需要保存模型，需要用以下方式：<br>
&nbsp;&nbsp;&nbsp;&nbsp;toch.save(model.state_dict(), "model.pth") #仅保存参数：weights & bias <br>
&nbsp;&nbsp;&nbsp;&nbsp;toch.save(model, "model.pth") #除了参数以外，还保存了模型的结构

In [ ]:
'''Evaluate the performance of the model'''
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X) 
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 1.with torch.no_grad()的作用
## 1.1 功能
（1）disable 梯度计算；
（2）节省内存和计算资源的消耗；
## 1.2 原理
模型不会将梯度存储在模型参数的.grad属性中。
## 1.3 同等操作
```python
for param in model.parameters()
    param.requires_grad = False
```

# 2.pred的格式是什么？
pred是模型输出的logits，shape是(batch_size, num_classes) <br>
```python
# Simulating model predictions (logits) for 4 samples & 3 classes
torch.tensor([[2.5, 1.2, 0.3],   # Class 0 (highest value at index 0)
                     [0.1, 3.2, 2.8],   # Class 1 (highest value at index 1)
                     [1.5, 2.1, 4.0],   # Class 2 (highest value at index 2)
                     [2.2, 3.9, 0.5]])  # Class 1 (highest value at index 1)
```

In [ ]:
'''Training process'''
epochs = 5
for t in range(epochs):
    print(f"Epoch:{t + 1}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")

In [ ]:
'''Saving and loading models'''
torch.save(model.state_dict(), "model.pth")

model =  NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only = True))

# 1.保存模型的不同方式
## 1.1 仅仅保存模型的参数
使用如上的方式保存和加载模型的参数，加载时需要保证模型的结构和存储的参数结构是一样的
## 1.2 同时保存参数和模型结构
```python
    torch.save(model, "model.pth")
    torch.load("model.pth")
    #这种方式就不需要先实例化一个未训练的模型了
```